In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')
all_data = pd.concat([train, test], axis=0)

In [3]:
all_data.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,<NA>,True Crime,74.81,Thursday,Night,<NA>,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.8,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.9,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.7,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


## Data Understanding

In [4]:
train.sample(10)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
24940,24940,Sports Weekly,Episode 94,15.24,Sports,33.83,Sunday,Night,95.14,2.0,Negative,2.49232
283885,283885,Market Masters,Episode 80,101.57,Business,81.92,Tuesday,Morning,10.98,3.0,Negative,63.93965
365444,365444,Current Affairs,Episode 37,62.10,News,39.38,Sunday,Morning,69.3,2.0,Positive,47.35639
17572,17572,Learning Lab,Episode 8,70.05,Education,34.87,Wednesday,Afternoon,<NA>,2.0,Positive,56.69859
679834,679834,Style Guide,Episode 85,14.48,Lifestyle,39.14,Monday,Night,50.72,1.0,Negative,13.79000
472284,472284,Detective Diaries,Episode 58,36.61,True Crime,79.96,Thursday,Afternoon,38.85,3.0,Positive,15.90571
137477,137477,Fitness First,Episode 99,117.60,Health,49.66,Friday,Night,74.84,2.0,Neutral,86.12590
631822,631822,Humor Hub,Episode 91,41.09,Comedy,92.10,Wednesday,Morning,82.86,3.0,Negative,22.73159
410561,410561,Detective Diaries,Episode 80,34.14,True Crime,35.63,Monday,Afternoon,50.88,1.0,Positive,13.50000
474238,474238,Finance Focus,Episode 16,97.78,Business,34.50,Saturday,Afternoon,15.84,2.0,Positive,64.03376


In [5]:
train.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,750000.000000,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,374999.500000,64.504738,59.859901,52.236449,1.348855,45.437406
std,216506.495284,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000
25%,187499.750000,35.730000,39.410000,28.380000,0.000000,23.178350
50%,374999.500000,63.840000,60.050000,53.580000,1.000000,43.379460
75%,562499.250000,94.070000,79.530000,76.600000,2.000000,64.811580
max,749999.000000,325.240000,119.460000,119.910000,103.910000,119.970000


In [6]:
train.shape

(750000, 12)

In [7]:
test.shape

(250000, 11)

In [8]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   id                           750000 non-null  int64
 1   Podcast_Name                 750000 non-null  object
 2   Episode_Title                750000 non-null  object
 3   Episode_Length_minutes       662907 non-null  float64
 4   Genre                        750000 non-null  object
 5   Host_Popularity_percentage   750000 non-null  float64
 6   Publication_Day              750000 non-null  object
 7   Publication_Time             750000 non-null  object
 8   Guest_Popularity_percentage  603970 non-null  float64
 9   Number_of_Ads                749999 non-null  float64
 10  Episode_Sentiment            750000 non-null  object
 11  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 88.8+ MB


In [9]:
train.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [10]:
test.isnull().sum()

id                                 0
Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
dtype: int64

## Data Preparation

In [11]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='median')
num_cols = test.select_dtypes(include=['float64']).columns
train[num_cols] = impute.fit_transform(train[num_cols])
test[num_cols] = impute.fit_transform(test[num_cols])

## Feature engineering

In [12]:
def add_features(df):
    df['Guest_Popularity_percentage'] =  df['Guest_Popularity_percentage'].clip(upper=100)
    df['Host_Popularity_percentage'] =  df['Host_Popularity_percentage'].clip(upper=100)
    df['Number_of_Ads'] = df['Number_of_Ads'].astype(int)
    df['title_number'] = df['Episode_Title'].str.split().str[1].astype(int)
    
    df['IsWeekend'] = df['Publication_Day'].isin(['Saturday', 'Sunday']).astype(int)
    
    sentiments = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
    df['Episode_Sentiment'] = df['Episode_Sentiment'].map(sentiments)
    
    df['Guest_Popularity_lengthmin'] =  df['Guest_Popularity_percentage'] * df['Episode_Length_minutes']
    df['Title_Length'] =  df['title_number'] * df['Episode_Length_minutes']
    df['Host_Popularity_lengthmin'] =  df['Host_Popularity_percentage'] * df['Episode_Length_minutes']
    df['Average_popularity'] = (df['Host_Popularity_percentage'] + df['Guest_Popularity_percentage']) / 2
    df['minutes_per_ads'] = df['Episode_Length_minutes'] / (df['Number_of_Ads'] + 1) 
    
    return df 

## Identifying outlier and clipping them

In [13]:
for colname in test.select_dtypes(exclude=['object', 'category']).columns:
    Q1 = all_data[colname].quantile(0.25)
    Q3 = all_data[colname].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = (all_data[colname] < lower_bound) | (all_data[colname] > upper_bound)
    
    train[colname] = train[colname].clip(lower=lower_bound, upper=upper_bound)
    test[colname] = test[colname].clip(lower=lower_bound, upper=upper_bound)

In [14]:
cat_cols = ['Podcast_Name','Genre', 'Publication_Day', 'Publication_Time']
for colname in cat_cols:
    train[colname] = train[colname].astype('category')
    test[colname] = test[colname].astype('category')

In [15]:
from sklearn.preprocessing import LabelEncoder


cat_cols = ['Podcast_Name','Genre', 'Publication_Day', 'Publication_Time']
for colname in cat_cols:
    le = LabelEncoder()
    train[colname] = le.fit_transform(train[colname])
    test[colname] = le.transform(test[colname])

## Splitting the train data by KFold

In [16]:
from sklearn.model_selection import KFold

X = train.copy()
y = X.pop('Listening_Time_minutes')
X = add_features(X)
X.drop('Episode_Title', axis=1, inplace=True)

X_test = test.copy()
X_test = add_features(X_test)
X_test.drop('Episode_Title', axis=1, inplace=True)

kf = KFold(n_splits=10, shuffle=True, random_state=600)

for train_index, valid_index in kf.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]  

## Target encoding

In [17]:
stats = ['mean', 'median', 'std', 'skew']

train_with_target = X_train.copy()
train_with_target['Listening_Time_minutes'] = y_train

grouped_stats = train_with_target.groupby(['Episode_Length_minutes'])['Listening_Time_minutes'].agg(stats).fillna(0.0)
grouped_stats.columns = [f'TE1_WC_{stat}' for stat in stats]

grouped_stats_full = train.groupby(['Episode_Length_minutes'])['Listening_Time_minutes'].agg(stats).fillna(0.0)
grouped_stats_full.columns = [f'TE1_WC_{stat}' for stat in stats]

X_train = X_train.merge(grouped_stats, on='Episode_Length_minutes', how='left')
X_valid = X_valid.merge(grouped_stats, on='Episode_Length_minutes', how='left')
X_test = X_test.merge(grouped_stats_full, on='Episode_Length_minutes', how='left')

In [18]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

xgb = XGBRegressor(
    n_estimators=386, 
    learning_rate=0.0798,
    verbose=0, 
    reg_lambda=8.2066, 
    reg_alpha= 9.2621, 
    max_depth=11,
    eval_metric='rmse',
    device='cuda'
).fit(X_train, y_train)

pred_xgb = xgb.predict(X_valid)
print(f'RMSE: {np.sqrt(mean_squared_error(y_valid, pred_xgb))}')


RMSE: 12.87480376802654


In [19]:
importances = xgb.feature_importances_
columns = X_train.columns

threshold = np.quantile(importances, 0.3)
selected_features = columns[importances > threshold]
top_X_train = X_train[selected_features]

print(selected_features)

Index(['Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage', 'Episode_Sentiment',
       'title_number', 'Guest_Popularity_lengthmin', 'Title_Length',
       'Host_Popularity_lengthmin', 'Average_popularity', 'minutes_per_ads',
       'TE1_WC_mean', 'TE1_WC_std'],
      dtype='object')


In [20]:
from sklearn.model_selection import cross_val_score

reduced_model_score = -1 * cross_val_score(xgb, top_X_train, y_train, scoring='neg_root_mean_squared_error').mean()
reduced_model_score

12.6655688668155

## Final Submission

In [21]:
expected_features = xgb.get_booster().feature_names
top_X_test_aligned = X_test[expected_features]
test_preds = xgb.predict(top_X_test_aligned)

output = pd.DataFrame({
    'id': test['id'],
    'Listening_Time_minutes': test_preds
})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
